In [1]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when
import json
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime
import csv
import pandas as pd
import pygeohash as pgh

In [2]:
#Set a global variable to store topic name.
topic_name = 'Assgn'

In [3]:
# Create a MongoClient
client = MongoClient()

# drop the collection climate_hotspot
#client.fit3182_db.climate_hotspot.drop()

# We will use the database: fit3182
db = client.fit3182_db

In [4]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)

In [5]:
topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('subscribe', topic_name)
    .load()
)

In [6]:
topic_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
output_stream_df = (
    topic_stream_df
    .select(                                      # 1
            topic_stream_df.value.cast('string')
        .alias('data')
    )
)

In [8]:
output_stream_df.printSchema()

root
 |-- data: string (nullable = true)



In [11]:
# reference from tutorial
def process_batch_data(batch_df, batch_id):
    data = batch_df.collect()
    hotspot_lst=[]
    climate={}
    for i in data:
        i=i.asDict()
        row=json.loads(i["data"])
        key=row['key']
        #print(key)
        if key=="climate":
            climate=row
        elif key=="aqua":
            hotspot_lst.append(row)
            #print('hello')
        elif key=="terra":
            hotspot_lst.append(row)
            #print('hello')
    #print(climate)
    latitude=[]
    time_lst=[]
    date_lst=[]
    confidence=[]
    longitude=[]
    surface_temperature_celcius=[]
    n_hotspot_lst=[]
    pghencoded=[]
    if climate!={}:
        date_val=climate['date']
        # pgh encoded value for climate data
        climate_pgh_encode=pgh.encode(climate['latitude'], climate['longitude'], precision=3)
        #print(climate['date'])
        hotspot=[]
        for h in hotspot_lst:
            hotspot_pgh_encode=pgh.encode(h['latitude'], h['longitude'], precision=3)
            
            # if match found for climate and hotspot pgh encoded value
            if hotspot_pgh_encode==climate_pgh_encode:
                # change the date to climate data's date
                h['date']=datetime.strptime(date_val, "%Y-%m-%d").date()          
                hotspot.append(h)
              
            
                
        
        # check if hotspot data is empty or not
        if len(hotspot)>0:
            for h in hotspot:
                # append hotspot data to resp list to later add to a dataframe
                latitude.append(float(h['latitude']))
                date_lst.append(str(h['date']))      
                time_lst.append(str(h['time']))
                longitude.append(float(h['longitude']))
                confidence.append(int(h['confidence']))
                surface_temperature_celcius.append(int(h['surface_temperature_celcius']))
                pghencoded.append(pgh.encode(h['latitude'], h['longitude'], precision=5))
    
    
            # create dataframe for the resp columns
            data_hotspot=pd.DataFrame([latitude, date_lst, time_lst, 
                                   longitude, confidence, 
                     surface_temperature_celcius, pghencoded]) 
        
            data_hotspot=data_hotspot.transpose()
            
            # column naming for data_hotspot dataframe
            data_hotspot.columns=['Latitude', 'Date Val', 'Time Val', 
                             'Longitude', 'Confidence', 
                              'Surface Temperature Celcius', 
                               'PGH Encode']
        
            
            # converting to float/integer data type 
            data_hotspot["Latitude"] = pd.to_numeric(data_hotspot["Latitude"], downcast="float")
            data_hotspot["Longitude"] = pd.to_numeric(data_hotspot["Longitude"], downcast="float")
            data_hotspot["Confidence"] = pd.to_numeric(data_hotspot["Confidence"], downcast="integer")
            data_hotspot["Surface Temperature Celcius"] = pd.to_numeric(data_hotspot["Surface Temperature Celcius"], downcast="integer")
        
        
        
            # grouping dataframe by pgh encoded value and applying averaging function on 
            # latitude, longitude, confidence, surface temperature celcius and creating a new df
            new_data_hotspot=data_hotspot.groupby('PGH Encode')[['Latitude',
                'Longitude', 'Confidence',
                'Surface Temperature Celcius']].agg('mean')
        
           
            # grouping dataframe by pgh encoded value and applying max function on 
            # date val and time val and creatng a new df
            another_data_hotspot=data_hotspot.groupby('PGH Encode')[['Date Val', 'Time Val']].agg('max')
        

        
        

                                  
            # convert dataframe to list
            # latitude, longitude, confidence, surface temperature celcius, pgh encode
            latitude=new_data_hotspot['Latitude'].values.tolist()
            longitude=new_data_hotspot['Longitude'].values.tolist()
            confidence=new_data_hotspot['Confidence'].values.tolist()
            surface_temperature_celcius=new_data_hotspot['Surface Temperature Celcius'].values.tolist()
            pgh_encode=new_data_hotspot.index.tolist()
        
            # 'Date Val', 'Time Val'
            date_val=another_data_hotspot['Date Val'].values.tolist()
            time_val=another_data_hotspot['Time Val'].values.tolist()
        
        
     
      
            n_hotspot_lst=[]
            for i in range(len(pgh_encode)):
                # tmp_doc
                tmp_doc = {
                "pgh encode": "",
                "avg confidence": 0,
                "date" :'20/04/2021',
                "time": '04:16:51',
                "avg latitude": 0.0,
                "avg longitude": 0.0,
                "avg surface_temperature_celcius": 0, 
                "cause_of_fire": "empty"
                }
                # adding updated hotspot info to dummy doc
                tmp_doc["pgh encode"]=pgh_encode[i]
                tmp_doc["avg latitude"]=latitude[i]
                tmp_doc["avg longitude"]=longitude[i]
                tmp_doc["avg confidence"]=confidence[i]
                tmp_doc["avg surface_temperature_celcius"]=surface_temperature_celcius[i]
                tmp_doc["date"]=date_val[i]
                tmp_doc["time"]=time_val[i]

            
                
                # check the cause of fire
                if int(climate['air_temperature_celcius'])>20 and int(climate['GHI_w/m2'])>180:
                    tmp_doc['cause_of_fire']='natural'
                else:
                    tmp_doc['cause_of_fire']='other'
            
                # adding the updated doc to n_hotspot_lst
                n_hotspot_lst=n_hotspot_lst+[tmp_doc]

                
        # update climate attributes
        climate['hotspot']=n_hotspot_lst
        pprint(climate)
        

        # The collection name is climate_hotspot
        climate_hotspot_collection= db.climate_hotspot   
        
        # adding climate doc to collection
        climate_hotspot_collection.insert_one(climate)
        
    return True

In [12]:
db_writer = (
    output_stream_df
    .writeStream
    .outputMode('append')
    .trigger(processingTime='10 seconds') # collect in 10 seconds
    .foreachBatch(process_batch_data)
)

In [13]:
writer=db_writer

In [14]:
try:
    query = writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop() 

{'GHI_w/m2': '134',
 'air_temperature_celcius': 15,
 'date': '2022-01-05',
 'hotspot': [],
 'key': 'climate',
 'latitude': -37.6008,
 'longitude': 143.0558,
 'max_wind_speed': 19.0,
 'precipitation': 0.0,
 'preciptation_form': 'I',
 'relative_humidity': 44.8,
 'windspeed_knots': 8.3}
hello
{'GHI_w/m2': '192',
 'air_temperature_celcius': 23,
 'date': '2022-01-06',
 'hotspot': [{'avg confidence': 81,
              'avg latitude': -37.61399841308594,
              'avg longitude': 143.4510040283203,
              'avg surface_temperature_celcius': 53,
              'cause_of_fire': 'natural',
              'date': '2022-01-06',
              'pgh encode': 'r1q40',
              'time': '19:12:00'}],
 'key': 'climate',
 'latitude': -37.621,
 'longitude': 143.447,
 'max_wind_speed': 11.1,
 'precipitation': 0.0,
 'preciptation_form': 'I',
 'relative_humidity': 53.4,
 'windspeed_knots': 6.1}
hello
{'GHI_w/m2': '124',
 'air_temperature_celcius': 14,
 'date': '2022-01-07',
 'hotspot': [{'avg co

hello
{'GHI_w/m2': '103',
 'air_temperature_celcius': 12,
 'date': '2022-01-26',
 'hotspot': [{'avg confidence': 60,
              'avg latitude': -36.95240020751953,
              'avg longitude': 143.75010681152344,
              'avg surface_temperature_celcius': 40,
              'cause_of_fire': 'other',
              'date': '2022-01-26',
              'pgh encode': 'r1qjz',
              'time': '14:24:00'}],
 'key': 'climate',
 'latitude': -37.288,
 'longitude': 144.39,
 'max_wind_speed': 13.0,
 'precipitation': 0.02,
 'preciptation_form': 'G',
 'relative_humidity': 49.9,
 'windspeed_knots': 6.1}
{'GHI_w/m2': '188',
 'air_temperature_celcius': 24,
 'date': '2022-01-27',
 'hotspot': [],
 'key': 'climate',
 'latitude': -36.0279,
 'longitude': 146.0244,
 'max_wind_speed': 13.0,
 'precipitation': 0.12,
 'preciptation_form': 'G',
 'relative_humidity': 60.1,
 'windspeed_knots': 8.0}
hello
{'GHI_w/m2': '111',
 'air_temperature_celcius': 13,
 'date': '2022-01-28',
 'hotspot': [{'avg co

hello
hello
{'GHI_w/m2': '111',
 'air_temperature_celcius': 13,
 'date': '2022-02-09',
 'hotspot': [{'avg confidence': 87,
              'avg latitude': -37.69200134277344,
              'avg longitude': 143.59300231933594,
              'avg surface_temperature_celcius': 62,
              'cause_of_fire': 'other',
              'date': '2022-02-09',
              'pgh encode': 'r1q1e',
              'time': '00:00:00'},
             {'avg confidence': 62,
              'avg latitude': -36.91709899902344,
              'avg longitude': 143.4748992919922,
              'avg surface_temperature_celcius': 41,
              'cause_of_fire': 'other',
              'date': '2022-02-09',
              'pgh encode': 'r1qjb',
              'time': '19:12:00'}],
 'key': 'climate',
 'latitude': -37.4437,
 'longitude': 143.4924,
 'max_wind_speed': 13.0,
 'precipitation': 0.0,
 'preciptation_form': 'I',
 'relative_humidity': 50.7,
 'windspeed_knots': 6.1}
hello
{'GHI_w/m2': '105',
 'air_temperature

{'GHI_w/m2': '139',
 'air_temperature_celcius': 16,
 'date': '2022-02-25',
 'hotspot': [],
 'key': 'climate',
 'latitude': -37.61,
 'longitude': 149.296,
 'max_wind_speed': 19.0,
 'precipitation': 0.0,
 'preciptation_form': 'G',
 'relative_humidity': 48.3,
 'windspeed_knots': 14.4}
hello
hello
hello
hello
{'GHI_w/m2': '149',
 'air_temperature_celcius': 19,
 'date': '2022-02-26',
 'hotspot': [{'avg confidence': 91,
              'avg latitude': -37.24700164794922,
              'avg longitude': 141.2779998779297,
              'avg surface_temperature_celcius': 68,
              'cause_of_fire': 'other',
              'date': '2022-02-26',
              'pgh encode': 'r1kkn',
              'time': '09:36:00'},
             {'avg confidence': 69,
              'avg latitude': -36.6703987121582,
              'avg longitude': 141.3197021484375,
              'avg surface_temperature_celcius': 45,
              'cause_of_fire': 'other',
              'date': '2022-02-26',
              'pg

{'GHI_w/m2': '153',
 'air_temperature_celcius': 20,
 'date': '2022-03-15',
 'hotspot': [],
 'key': 'climate',
 'latitude': -34.965,
 'longitude': 142.031,
 'max_wind_speed': 15.0,
 'precipitation': 0.0,
 'preciptation_form': 'I',
 'relative_humidity': 62.6,
 'windspeed_knots': 10.1}
hello
hello
hello
{'GHI_w/m2': '141',
 'air_temperature_celcius': 17,
 'date': '2022-03-16',
 'hotspot': [{'avg confidence': 69,
              'avg latitude': -37.8130989074707,
              'avg longitude': 143.11749267578125,
              'avg surface_temperature_celcius': 45,
              'cause_of_fire': 'other',
              'date': '2022-03-16',
              'pgh encode': 'r1mbb',
              'time': '00:00:00'},
             {'avg confidence': 58,
              'avg latitude': -37.73789978027344,
              'avg longitude': 143.17059326171875,
              'avg surface_temperature_celcius': 40,
              'cause_of_fire': 'other',
              'date': '2022-03-16',
              'pgh e

...

hello
hello
{'GHI_w/m2': '140',
 'air_temperature_celcius': 18,
 'date': '2022-11-23',
 'hotspot': [{'avg confidence': 89,
              'avg latitude': -37.744598388671875,
              'avg longitude': 143.4084930419922,
              'avg surface_temperature_celcius': 66,
              'cause_of_fire': 'other',
              'date': '2022-11-23',
              'pgh encode': 'r1mcr',
              'time': '04:48:00'},
             {'avg confidence': 72,
              'avg latitude': -37.58110046386719,
              'avg longitude': 143.12969970703125,
              'avg surface_temperature_celcius': 46,
              'cause_of_fire': 'other',
              'date': '2022-11-23',
              'pgh encode': 'r1mf0',
              'time': '14:24:00'}],
 'key': 'climate',
 'latitude': -36.701,
 'longitude': 142.538,
 'max_wind_speed': 11.1,
 'precipitation': 0.28,
 'preciptation_form': 'G',
 'relative_humidity': 61.5,
 'windspeed_knots': 5.5}
hello
{'GHI_w/m2': '142',
 'air_temperature

hello
hello
hello
{'GHI_w/m2': '142',
 'air_temperature_celcius': 17,
 'date': '2022-12-06',
 'hotspot': [{'avg confidence': 92,
              'avg latitude': -37.64979934692383,
              'avg longitude': 142.59970092773438,
              'avg surface_temperature_celcius': 70,
              'cause_of_fire': 'other',
              'date': '2022-12-06',
              'pgh encode': 'r1m3u',
              'time': '09:36:00'},
             {'avg confidence': 96,
              'avg latitude': -37.75419998168945,
              'avg longitude': 142.99490356445312,
              'avg surface_temperature_celcius': 77,
              'cause_of_fire': 'other',
              'date': '2022-12-06',
              'pgh encode': 'r1m9j',
              'time': '00:00:00'},
             {'avg confidence': 85,
              'avg latitude': -37.713401794433594,
              'avg longitude': 143.38180541992188,
              'avg surface_temperature_celcius': 59,
              'cause_of_fire': 'other',


hello
{'GHI_w/m2': '117',
 'air_temperature_celcius': 13,
 'date': '2022-12-20',
 'hotspot': [{'avg confidence': 92,
              'avg latitude': -37.81610107421875,
              'avg longitude': 142.92860412597656,
              'avg surface_temperature_celcius': 71,
              'cause_of_fire': 'other',
              'date': '2022-12-20',
              'pgh encode': 'r1m8u',
              'time': '04:48:00'}],
 'key': 'climate',
 'latitude': -36.7084,
 'longitude': 142.7354,
 'max_wind_speed': 19.0,
 'precipitation': 0.02,
 'preciptation_form': 'G',
 'relative_humidity': 44.1,
 'windspeed_knots': 12.9}
{'GHI_w/m2': '195',
 'air_temperature_celcius': 25,
 'date': '2022-12-21',
 'hotspot': [],
 'key': 'climate',
 'latitude': -36.404,
 'longitude': 142.5467,
 'max_wind_speed': 8.9,
 'precipitation': 0.0,
 'preciptation_form': 'I',
 'relative_humidity': 60.9,
 'windspeed_knots': 6.3}
{'GHI_w/m2': '92',
 'air_temperature_celcius': 10,
 'date': '2022-12-22',
 'hotspot': [],
 'key': 'cl

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/student/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/student/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopped query
